In [4]:
import pandas as pd
import os

# Folder setup
data_dir = "energyData"
cleaned_dir = "energyData_clean"
os.makedirs(cleaned_dir, exist_ok=True)

# Only process pjm_hourly_est.csv
filepath = os.path.join(data_dir, "pjm_hourly_est.csv")
df = pd.read_csv(filepath)
print(f"Processing pjm_hourly_est.csv...")
print("Columns:", df.columns.tolist())

datetime_col = 'Datetime' if 'Datetime' in df.columns else df.columns[0]

# Process each region column
for region_col in df.columns:
    if region_col == datetime_col:
        continue

    df_region = df[[datetime_col, region_col]].dropna()
    df_region[datetime_col] = pd.to_datetime(df_region[datetime_col], errors='coerce')
    df_region.set_index(datetime_col, inplace=True)
    df_region = df_region.sort_index()

    df_daily = df_region.resample('D').mean()
    df_daily.dropna(inplace=True)

    # Normalize PJM_Load → PJM
    region_name = "PJM" if region_col == "PJM_Load" else region_col
    cleaned_path = os.path.join(cleaned_dir, f"{region_name}_MW_hourly_daily.csv")
    df_daily.to_csv(cleaned_path)
    print(f"Saved cleaned file to: {cleaned_path}")


Processing pjm_hourly_est.csv...
Columns: ['Datetime', 'AEP', 'COMED', 'DAYTON', 'DEOK', 'DOM', 'DUQ', 'EKPC', 'FE', 'NI', 'PJME', 'PJMW', 'PJM_Load']
Saved cleaned file to: energyData_clean\AEP_MW_hourly_daily.csv
Saved cleaned file to: energyData_clean\COMED_MW_hourly_daily.csv
Saved cleaned file to: energyData_clean\DAYTON_MW_hourly_daily.csv
Saved cleaned file to: energyData_clean\DEOK_MW_hourly_daily.csv
Saved cleaned file to: energyData_clean\DOM_MW_hourly_daily.csv
Saved cleaned file to: energyData_clean\DUQ_MW_hourly_daily.csv
Saved cleaned file to: energyData_clean\EKPC_MW_hourly_daily.csv
Saved cleaned file to: energyData_clean\FE_MW_hourly_daily.csv
Saved cleaned file to: energyData_clean\NI_MW_hourly_daily.csv
Saved cleaned file to: energyData_clean\PJME_MW_hourly_daily.csv
Saved cleaned file to: energyData_clean\PJMW_MW_hourly_daily.csv
Saved cleaned file to: energyData_clean\PJM_MW_hourly_daily.csv


In [5]:
file_list = [f for f in os.listdir(cleaned_dir) if f.endswith("_daily.csv")]

for filename in file_list:
    filepath = os.path.join(cleaned_dir, filename)
    print(f"\n🔍 Validating {filename}...")

    try:
        df = pd.read_csv(filepath, index_col=0, parse_dates=True)

        # Basic checks
        assert isinstance(df.index, pd.DatetimeIndex), "❌ Index is not datetime"
        assert df.isnull().sum().sum() == 0, "❌ Contains missing values"

        # Range checks
        col = df.columns[0]
        if not (5000 < df[col].min() and df[col].max() < 100000):
            print(f"⚠️  {col} values out of expected MW range: min={df[col].min()}, max={df[col].max()}")

        # Print info
        print(f"✅ {filename}:")
        print(f"   Rows: {len(df)}")
        print(f"   Date Range: {df.index.min().date()} → {df.index.max().date()}")
        print(f"   MW Range: {df[col].min():.2f} → {df[col].max():.2f}")

    except Exception as e:
        print(f"❌ Error in {filename}: {e}")


🔍 Validating AEP_MW_hourly_daily.csv...
✅ AEP_MW_hourly_daily.csv:
   Rows: 5055
   Date Range: 2004-10-01 → 2018-08-03
   MW Range: 11078.04 → 22847.88

🔍 Validating COMED_MW_hourly_daily.csv...
✅ COMED_MW_hourly_daily.csv:
   Rows: 2772
   Date Range: 2011-01-01 → 2018-08-03
   MW Range: 8148.75 → 19920.29

🔍 Validating DAYTON_MW_hourly_daily.csv...
⚠️  DAYTON values out of expected MW range: min=1366.3333333333333, max=3136.625
✅ DAYTON_MW_hourly_daily.csv:
   Rows: 5055
   Date Range: 2004-10-01 → 2018-08-03
   MW Range: 1366.33 → 3136.62

🔍 Validating DEOK_MW_hourly_daily.csv...
⚠️  DEOK values out of expected MW range: min=1219.0, max=4503.458333333333
✅ DEOK_MW_hourly_daily.csv:
   Rows: 2407
   Date Range: 2012-01-01 → 2018-08-03
   MW Range: 1219.00 → 4503.46

🔍 Validating DOM_MW_hourly_daily.csv...
✅ DOM_MW_hourly_daily.csv:
   Rows: 4843
   Date Range: 2005-05-01 → 2018-08-03
   MW Range: 7772.00 → 18976.62

🔍 Validating DUQ_MW_hourly_daily.csv...
⚠️  DUQ values out of expe